In [ ]:
import pandas as pd
file = "G:/My Drive/SHU MS/2nd Sem/TEXT-BASED ANALYSIS CS-651-A/Week 2/un-general-debates-blueprint.csv"
df = pd.read_csv(file)
df.head(20)

In [ ]:
df = df.sample(frac=0.2) # only for large datasets
df

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['length'] = df['text'].str.len()
df.describe()

In [ ]:
df.describe(include='O')

In [ ]:
df.isna().sum()

In [ ]:
df['speaker'].fillna('unknown', inplace=True)
df['position'].fillna('unknown', inplace=True)
df.isna().sum()

In [ ]:
df['speaker'] = df['speaker'].str.lower()
df['position'] = df['position'].str.lower()
df['text'] = df['text'].str.lower()
df['country'] = df['country'].str.lower()
df['country_name'] = df['country_name'].str.lower()
df.head(10)

In [ ]:
df[df['speaker'].str.contains('bush')]

In [ ]:
df['length'].plot()

In [ ]:
df['length'].plot(kind='box', vert=False)

In [ ]:
df['length'].plot(kind='hist', bins=30)

In [ ]:
import seaborn as sns 
where = df['country'].isin(['usa', 'fra', 'gbr', 'chn', 'rus','ind','egy'])
g = sns.catplot(data=df[where], x="country", y="length", kind='box')
g.fig.set_size_inches(10, 8) ###

In [ ]:
import regex as re

def tokenize(text):
    return re.findall(r'[\w-]*[a-z|A-Z][\w-]*', text)

In [ ]:
testtext = "Let's defeat SARS-CoV-2 together in 2020! www.cdc.gov"
testtokens = tokenize(testtext)
 
print(testtokens)

In [ ]:
import nltk 
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))
print(stopwords)

In [ ]:
def remove_stop(tokens):
    return [t for t in tokens if t.lower() not in stopwords]
#Just testing it
cleantokens= remove_stop(testtokens)
print(cleantokens)

In [ ]:
pipeline = [str.lower, tokenize, remove_stop]

In [ ]:
def prepare(text, pipeline):
    tokens = text
    for fun in pipeline:
        tokens = fun(tokens)
    return tokens

In [ ]:
df['tokens'] = df['text'].apply(prepare, pipeline=pipeline)

In [ ]:
df.head(1)

In [ ]:
df['num_tokens'] = df['tokens'].map(len)
df.tail()

In [ ]:
from collections import Counter

tokens = tokenize("She likes my cats and my cats like my sofa.")

testcount= Counter(tokens)
print(testcount)

In [ ]:
more_tokens = tokenize("She likes dogs and cats.")
testcount.update(more_tokens)
print(testcount)

In [ ]:
counter = Counter()
df['tokens'].map(counter.update)
print(counter.most_common(10))
print(counter)

In [ ]:
freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
freq_df = freq_df.query('freq >= 1')
freq_df.index.name = 'token'

In [ ]:
freq_df.head(5)

In [ ]:
freq_df = freq_df.sort_values('freq', ascending=False)
freq_df.head(10)

In [ ]:
freq_df.describe()

In [ ]:
ax = freq_df.head(50).plot(kind='barh', width=0.95, figsize=(12,8))
ax.invert_yaxis()
ax.set(xlabel='Frequency', ylabel='Token', title='Top Words')

In [ ]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

text = df.query("year==2015 and country=='ind'")['text'].values[0]

plt.figure(figsize=(14, 7)) ###
wc = WordCloud(max_words=100, stopwords=stopwords)
wc.generate(text)
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")

In [ ]:
wc = WordCloud(background_color='white', width = 300, height=300, max_words=100, margin=2)
wc.fit_words(freq_df['freq'].to_dict() )
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wc)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()